In [1]:
import time
import numpy as np
from opt_einsum import contract as einsum
import pyscf
from pyscf import gto, scf, dft, tddft, data, lib
import argparse
import os
import yaml
from pyscf.tools import molden
from pyscf.dft import xcfun

In [ ]:
# ###########################################################################
# # build geometry in PySCF
# mol = gto.Mole()
# mol.build(atom = '\
# C         -4.63649        2.70349        0.00000;\
# C         -3.11729        2.70349        0.00000;\
# H         -5.01303        2.36410        0.98791;\
# H         -5.01303        2.01763       -0.78787;\
# H         -5.01303        3.72874       -0.20003;\
# H         -2.74074        1.67824        0.20003;\
# H         -2.74074        3.38934        0.78788;\
# H         -2.74074        3.04288       -0.98791',\
# basis = 'def2-SVP')
# ###########################################################################

In [59]:
mol = gto.Mole()
mol.build(atom = '\
C         -1.44673        2.80824       -0.07813;\
O         -1.78998        3.80792       -0.69188;\
N         -1.18291        1.66325       -0.74926;\
N         -1.33221        2.85136        1.26946;\
H         -1.53168        3.72026        1.78651;\
H         -1.04365        2.01512        1.79805;\
H         -1.26824        1.62382       -1.77554;\
H         -0.89270        0.81388       -0.24298'\
, basis = 'def2-SVP')

In [60]:
mf = dft.RKS(mol) 
mf.conv_tol = 1e-9
mf.grids.level = 3     # 0-9, big number for large mesh grids, default is 3

mf = mf.density_fit()
mf.xc = 'wb97x'


kernel_0 = time.time()
mf.kernel()
kernel_1 = time.time()
kernel_t = kernel_1 - kernel_0
print ('SCF Done after ', round(kernel_t, 4), 'seconds')

converged SCF energy = -225.042009112821
SCF Done after  5.2917 seconds


In [61]:
mo_occ = mf.mo_occ



########################################################################
# Collect everything needed from PySCF
Qstart = time.time()
# extract vind() function
td = tddft.TDA(mf)

vind, hdiag = td.gen_vind(mf)

# vind (V) = A*V
def matrix_vector(V):
    return vind(V.T).T

Natm = mol.natm


occupied = len(np.where(mo_occ > 0)[0])
#mf.mo_occ is an array of occupance [2,2,2,2,2,0,0,0,0.....]
virtual = len(np.where(mo_occ == 0)[0])

# AO = [int(i.split(' ',1)[0]) for i in mol.ao_labels()]
# # .split(' ',1) is to split each element by space, split once.
# # mol.ao_labels() it is Labels of AO basis functions
# # AO is a list of corresponding atom_id

N_bf = len(mo_occ)
R = pyscf.gto.mole.inter_distance(mol, coords=None)
#Inter-particle distance array
# unit == ’Bohr’, Its value is 5.29177210903(80)×10^(−11) m
########################################################################


##################################################################################################
# create a function for dictionary of chemical hardness, by mappig two iteratable subject, list
# list of elements



elements = ['H' , 'He', 'Li', 'Be', 'B' , 'C' , 'N' , 'O' , 'F' , 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P' , 'S' , 'Cl', 'Ar', 'K' , 'Ca',
    'Sc', 'Ti', 'V' , 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y' , 'Zr',
    'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I' , 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W' , 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
    'Pa', 'U' , 'Np', 'Pu']
#list of chemical hardness, they are floats, containing elements 1-94, in Hartree
hardness = [
0.47259288,
0.92203391,
0.17452888,
0.25700733,
0.33949086,
0.42195412,
0.50438193,
0.58691863,
0.66931351,
0.75191607,
0.17964105,
0.22157276,
0.26348578,
0.30539645,
0.34734014,
0.38924725,
0.43115670,
0.47308269,
0.17105469,
0.20276244,
0.21007322,
0.21739647,
0.22471039,
0.23201501,
0.23933969,
0.24665638,
0.25398255,
0.26128863,
0.26859476,
0.27592565,
0.30762999,
0.33931580,
0.37235985,
0.40273549,
0.43445776,
0.46611708,
0.15585079,
0.18649324,
0.19356210,
0.20063311,
0.20770522,
0.21477254,
0.22184614,
0.22891872,
0.23598621,
0.24305612,
0.25013018,
0.25719937,
0.28784780,
0.31848673,
0.34912431,
0.37976593,
0.41040808,
0.44105777,
0.05019332,
0.06762570,
0.08504445,
0.10247736,
0.11991105,
0.13732772,
0.15476297,
0.17218265,
0.18961288,
0.20704760,
0.22446752,
0.24189645,
0.25932503,
0.27676094,
0.29418231,
0.31159587,
0.32902274,
0.34592298,
0.36388048,
0.38130586,
0.39877476,
0.41614298,
0.43364510,
0.45104014,
0.46848986,
0.48584550,
0.12526730,
0.14268677,
0.16011615,
0.17755889,
0.19497557,
0.21240778,
0.07263525,
0.09422158,
0.09920295,
0.10418621,
0.14235633,
0.16394294,
0.18551941,
0.22370139]
HARDNESS = dict(zip(elements,hardness))
#function to return chemical hardness from dictionary HARDNESS
def Hardness (atom_id):
    atom = mol.atom_pure_symbol(atom_id)
    return HARDNESS[atom]
# mol.atom_pure_symbol(atom_id) returns pure element symbol, no special characters


########################################################################
# This block is the function to produce orthonormalized coefficient matrix C
def matrix_power (S,a):
    s,ket = np.linalg.eigh(S)
    s = s**a
    X = np.linalg.multi_dot([ket,np.diag(s),ket.T])
    #X == S^1/2
    return X

def orthonormalize (C):
    X = matrix_power(mf.get_ovlp(), 0.5)
    # S = mf.get_ovlp() #.get_ovlp() is basis overlap matrix
    # S = np.dot(np.linalg.inv(c.T), np.linalg.inv(c))
    C = np.dot(X,C)
    return C

C = mf.mo_coeff
# mf.mo_coeff is the coefficient matrix

C = orthonormalize (C)
# C is orthonormalized coefficient matrix
# np.dot(C.T,C) is a an identity matrix
########################################################################
RSH_F = [
'lc-b3lyp',
'wb97',
'wb97x',
'wb97x-d3',
'cam-b3lyp']
RSH_paramt = [
[0.53, 8.00, 4.50],
[0.61, 8.00, 4.41],
[0.56, 8.00, 4.58],
[0.51, 8.00, 4.51],
[0.38, 1.86, 0.90]]
RSH_F_paramt = dict(zip(RSH_F, RSH_paramt))

hybride_F = [
'b3lyp',
'tpssh',
'm05-2x',
'pbe0',
'm06',
'm06-2x',
'NA']# NA is for Hartree-Fork
hybride_paramt = [0.2, 0.1, 0.56, 0.25, 0.27, 0.54, 1]
DF_ax = dict(zip(hybride_F, hybride_paramt))
#Zhao, Y. and Truhlar, D.G., 2006. Density functional for spectroscopy: no long-range self-interaction error, good performance for Rydberg and charge-transfer states, and better performance on average than B3LYP for ground states. The Journal of Physical Chemistry A, 110(49), pp.13126-13130.

if mf.xc in RSH_F:
    a_x, beta, alpha = RSH_F_paramt[mf.xc]

elif mf.xc in hybride_F:
    beta1 = 0.2
    beta2 = 1.83
    alpha1 = 1.42
    alpha2 = 0.48

    a_x = DF_ax[mf.xc]
    beta = beta1 + beta2 * a_x
    alpha = alpha1 + alpha2 * a_x


# creat \eta matrix
a = [Hardness (atom_id) for atom_id in range (Natm)]
a = np.asarray(a).reshape(1,-1)
eta = (a+a.T)/2

# creat GammaK and GammaK matrix
GammaJ = (R**beta + (a_x * eta)**(-beta))**(-1/beta)
GammaK = (R**alpha + eta**(-alpha)) **(-1/alpha)


Natm = mol.natm
def generateQ ():
    aoslice = mol.aoslice_by_atom()
    q = np.zeros([Natm, N_bf, N_bf])
    #N_bf is number Atomic orbitals, occupied+virtual, q is same size with C
    for atom_id in range (0, Natm):
        shst, shend, atstart, atend = aoslice[atom_id]
        q[atom_id,:, :] = np.dot(C[atstart:atend, :].T, C[atstart:atend, :])
    return q

q_tensors = generateQ()


q_tensor_ij = np.zeros((Natm, occupied, occupied))
q_tensor_ij[:,:,:] = q_tensors[:, :occupied,:occupied]

q_tensor_ab = np.zeros((Natm, virtual, virtual))
q_tensor_ab[:,:,:] = q_tensors[:, occupied:,occupied:]

q_tensor_ia = np.zeros((Natm, occupied, virtual))
q_tensor_ia[:,:,:] = q_tensors[:, :occupied,occupied:]


Q_K = einsum('Bjb, AB -> Ajb', q_tensor_ia, GammaK)
Q_J = einsum('Bab, AB -> Aab', q_tensor_ab, GammaJ)
# pre-calculate and store the Q-Gamma rank 3 tensor
Qend = time.time()

Q_time = Qend - Qstart
print ('Q-Gamma tensors building time =', round(Q_time, 4))
##################################################################################################


###################################################################################################
# This block is to define on-the-fly two electron intergeral (pq|rs)
# A_iajb * v = delta_ia_ia*v + 2(ia|jb)*v - (ij|ab)*v

# iajb_v = einsum('Aia, Bjb, AB, jbm -> iam', q_tensor_ia, q_tensor_ia, GammaK, V)
# ijab_v = einsum('Aij, Bab, AB, jbm -> iam', q_tensor_ij, q_tensor_ab, GammaJ, V)

def iajb_fly (V):
    V = V.reshape(occupied, virtual, -1)
    Q_K_V = einsum('Ajb, jbm -> Am', Q_K, V)
    iajb_V = einsum('Aia, Am -> iam', q_tensor_ia, Q_K_V).reshape(occupied*virtual, -1)

    return iajb_V

def ijab_fly (V):
    V = V.reshape(occupied, virtual, -1)
    # (-1, occupied, virtual)
#     ijab_v = einsum('Aij, Aab, jbm -> iam', q_tensor_ij, Q_J,  V)

    # contract smaller index first
    # Aij_V = einsum('Aij, jbm -> Aibm', q_tensor_ij, V)
    # ijab_V = einsum('Aab, Aibm -> iam', Q_J, Aij_V).reshape(occupied*virtual, -1)

    # contract larger index first
    Aab_V = einsum('Aab, jbm -> jAam', Q_J, V)
    #('Aab, mjb -> mjaA')
    ijab_V = einsum('Aij, jAam -> iam', q_tensor_ij, Aab_V).reshape(occupied*virtual, -1)
    #('Aij, mjaA -> mia)
    return ijab_V

delta_diag_A = hdiag.reshape(occupied, virtual)



def delta_fly (V):
    V = V.reshape(occupied, virtual, -1)
    delta_v = einsum('ia,iam -> iam', delta_diag_A, V).reshape(occupied*virtual, -1)
    return delta_v

def sTDA_fly (V):
    # sTDA_A * V
    V = V.reshape(occupied*virtual,-1)
    # this feature can deal with multiple vectors
    sTDA_V =  delta_fly (V) + 2*iajb_fly (V) - ijab_fly (V)
    return sTDA_V
###################################################################################################

Q-Gamma tensors building time = 0.3451


In [62]:
##############################################################################################
# orthonormalization of guess_vectors
def Gram_Schdmit_bvec (A, bvec):
    # suppose A is orthonormalized
    projections_coeff = np.dot(A.T, bvec)
    bvec = bvec - np.dot(A, projections_coeff)
    return bvec

def Gram_Schdmit (A):
    # A matrix has J columns, orthonormalize each columns
    # unualified vectors will be removed
    N_rows = np.shape(A)[0]
    N_vectors = np.shape(A)[1]
    A = A/np.linalg.norm(A, axis=0, keepdims = True)

    B = np.zeros((N_rows,N_vectors))
    count = 0
    ############b
    for j in range (0, N_vectors):
        bvec = Gram_Schdmit_bvec (B[:, :count], A[:, j])
        norm = np.linalg.norm(bvec)
        if norm > 1e-14:
            B[:, count] = bvec/np.linalg.norm(bvec)
            count +=1
    return B[:, :count]

def Gram_Schdmit_fill_holder (V, count, vecs):
    # V is a vectors holder
    # count is the amount of vectors that already sit in the holder

    nvec = np.shape(vecs)[1]
    # amount of new vectors intended to fill in the V

    # count will be final amount of vectors in V
    for j in range (0, nvec):
        vec = vecs[:, j]
        vec = Gram_Schdmit_bvec(V[:, :count], vec)   #single orthonormalize
        vec = Gram_Schdmit_bvec(V[:, :count], vec) #double orthonormalize

        norm = np.linalg.norm(vec)
        if  norm > 1e-14:
            vec = vec/norm
            V[:, count] = vec
            count += 1
    new_count = count

    return V, new_count
########################################################################


####################################################################
# define the orthonormality of a matrix A as the norm of (A.T*A - I)
def check_orthonormal (A):
    n = np.shape(A)[1]
    B = np.dot (A.T, A)
    c = np.linalg.norm(B - np.eye(n))
    return c
####################################################################

In [63]:
########################################################################
def solve_AX_Xla_B (sub_A, eigen_lambda, sub_B):
    # AX - XB  = Q
    N_vectors = len(eigen_lambda)
    a, u = np.linalg.eigh(sub_A)
    ub = np.dot(u.T, sub_B)
    ux = np.zeros_like(sub_B)
    for k in range (N_vectors):
        ux[:, k] = ub[:, k]/(a - eigen_lambda[k])
    sub_guess = np.dot(u, ux)
    return sub_guess
#########################################################################

########################################################################
# K_inv
def K_inv (B, eigen_lambda):
    # exacty the same function with sTDA_preconditioner, just no dic
    # to solve K^(-1)y and K^(-1)u
    # (sTDA_A - eigen_lambda*I)^-1 B = X
    # AX - X\lambda = B
    # columns in B are residuals or current guess 
    precondition_start = time.time()

    N_rows = np.shape(B)[0]
    B = B.reshape(N_rows, -1)
    N_vectors = np.shape(B)[1]

    #number of vectors to be preconditioned
    bnorm = np.linalg.norm(B, axis=0, keepdims = True)
    #norm of each vectors in B, shape (1,-1)
    B = B/bnorm

    start = time.time()
    tol = 1e-2    # Convergence tolerance
    max = 30   # Maximum number of iterations

    V = np.zeros((N_rows, (max+1)*N_vectors))
    W = np.zeros((N_rows, (max+1)*N_vectors))
    count = 0

    # now V and W are empty holders, 0 vectors
    # W = sTDA_fly(V)
    # count is the amount of vectors that already sit in the holder
    # in each iteration, V and W will be filled/updated with new guess vectors

    ###########################################
    #initial guess: (diag(A) - \lambda)^-1 B.
    # D is preconditioner for each state
    t = 1e-10
    D = np.repeat(hdiag.reshape(-1,1), N_vectors, axis=1) - eigen_lambda
    D= np.where( abs(D) < t, np.sign(D)*t, D) # <t: returns np.sign(D)*t; else: D
    inv_D = 1/D

    # generate initial guess
    init = B*inv_D
    V, new_count = Gram_Schdmit_fill_holder (V, count, init)
    W[:, count:new_count] = sTDA_fly(V[:, count:new_count])
    count = new_count
    ####################################################################################
    for i in range (0, max):
        sub_B = np.dot(V[:,:count].T, B)
        sub_A = np.dot(V[:,:count].T, W[:,:count])
        #project sTDA_A matrix and vector B into subspace
        # size of subspace
        m = np.shape(sub_A)[0]
        sub_guess = solve_AX_Xla_B(sub_A, eigen_lambda, sub_B)
        full_guess = np.dot(V[:,:count], sub_guess)
        residual = np.dot(W[:,:count], sub_guess) - full_guess*eigen_lambda - B
        Norms_of_r = np.linalg.norm (residual, axis=0, keepdims = False)
        max_norm = np.max(Norms_of_r)

        if max_norm < tol:
            break

        # index for unconverged residuals
        index = [i for i in range(len(Norms_of_r)) if Norms_of_r[i] > tol]

        # preconditioning step
        # only generate new guess from unconverged residuals
        new_guess = residual[:,index]*inv_D[:,index]

        V, new_count = Gram_Schdmit_fill_holder (V, count, new_guess)
        W[:, count:new_count] = sTDA_fly(V[:, count:new_count])
        count = new_count

    precondition_end = time.time()
    precondition_time = precondition_end - precondition_start
    if i == (max -1):
        print ('_________________ sTDA Preconditioner Failed Due to Iteration Limit _________________')
        print ('sTDA preconditioning failed after ', i, 'steps; ', round(precondition_time, 4), 'seconds')
        print ('current residual norms', Norms_of_r)
        print ('max_norm = ', max_norm)
        print ('orthonormality of V', check_orthonormal(V[:,:count]))
    else:
        print ('K inverse Done after ', i, 'steps; ', round(precondition_time, 4), 'seconds')
    return (full_guess*bnorm)
###########################################################################################

In [70]:
############################
###########################
def Jacobi_preconditioner(B, eigen_lambda, current_dic, current_guess):
    # (1-uu*)(A-λ*I)(1-uu*)t = -B 
    # B is residual, we want to solve "t"
    # z approximates t
    # z = (A-λ*I)^(-1)*(-B) + α(A-λ*I)^(-1) * u
    # where α = [u*(A-λ*I)^(-1)y]/[u*(A-λ*I)^(-1)u]
    # first is to solve (A-λ*I)^(-1)y and (A-λ*I)^(-1)u 
    u = current_guess
    K_inv_y = K_inv(-B, eigen_lambda)
    K_inv_u = K_inv(current_guess, eigen_lambda)
    n = np.multiply(u, K_inv_y).sum(axis=0)
    d = np.multiply(u, K_inv_u).sum(axis=0)
    Alpha = n/d

    z = K_inv_y -  Alpha*K_inv_u
    return (z, current_dic)
############################
###########################

In [71]:
########################################################################
def solve_AX_Xla_B (sub_A, eigen_lambda, sub_B):
    # AX - XB  = Q
    N_vectors = len(eigen_lambda)
    a, u = np.linalg.eigh(sub_A)
    ub = np.dot(u.T, sub_B)
    ux = np.zeros_like(sub_B)
    for k in range (N_vectors):
        ux[:, k] = ub[:, k]/(a - eigen_lambda[k])
    sub_guess = np.dot(u, ux)
    return sub_guess
#########################################################################

########################################################################
# sTDA preconditioner
def on_the_fly_sTDA_preconditioner (B, eigen_lambda, current_dic, current_guess=None):
    # (sTDA_A - eigen_lambda*I)^-1 B = X
    # AX - X\lambda = B
    # columns in B are residuals (in Davidson's loop) to be preconditioned,
    precondition_start = time.time()

    N_rows = np.shape(B)[0]
    B = B.reshape(N_rows, -1)
    N_vectors = np.shape(B)[1]


    #number of vectors to be preconditioned
    bnorm = np.linalg.norm(B, axis=0, keepdims = True)
    #norm of each vectors in B, shape (1,-1)
    B = B/bnorm

    start = time.time()
    tol = 1e-2    # Convergence tolerance
    max = 30   # Maximum number of iterations

    V = np.zeros((N_rows, (max+1)*N_vectors))
    W = np.zeros((N_rows, (max+1)*N_vectors))
    count = 0

    # now V and W are empty holders, 0 vectors
    # W = sTDA_fly(V)
    # count is the amount of vectors that already sit in the holder
    # in each iteration, V and W will be filled/updated with new guess vectors

    ###########################################
    #initial guess: (diag(A) - \lambda)^-1 B.
    # D is preconditioner for each state
    t = 1e-10
    D = np.repeat(hdiag.reshape(-1,1), N_vectors, axis=1) - eigen_lambda
    D= np.where( abs(D) < t, np.sign(D)*t, D) # <t: returns np.sign(D)*t; else: D
    inv_D = 1/D

    # generate initial guess
    init = B*inv_D
    V, new_count = Gram_Schdmit_fill_holder (V, count, init)
    W[:, count:new_count] = sTDA_fly(V[:, count:new_count])
    count = new_count

    current_dic['preconditioning'] = []
    ####################################################################################
    for i in range (0, max):
        sub_B = np.dot(V[:,:count].T, B)
        sub_A = np.dot(V[:,:count].T, W[:,:count])
        #project sTDA_A matrix and vector B into subspace

        # size of subspace
        m = np.shape(sub_A)[0]

        sub_guess = solve_AX_Xla_B(sub_A, eigen_lambda, sub_B)

        full_guess = np.dot(V[:,:count], sub_guess)
        residual = np.dot(W[:,:count], sub_guess) - full_guess*eigen_lambda - B

        Norms_of_r = np.linalg.norm (residual, axis=0, keepdims = False)

        current_dic['preconditioning'].append({'precondition residual norms': Norms_of_r.tolist()})

        max_norm = np.max(Norms_of_r)

        if max_norm < tol:
            break

        # index for unconverged residuals
        index = [i for i in range(len(Norms_of_r)) if Norms_of_r[i] > tol]

        # preconditioning step
        # only generate new guess from unconverged residuals
        new_guess = residual[:,index]*inv_D[:,index]

        V, new_count = Gram_Schdmit_fill_holder (V, count, new_guess)
        W[:, count:new_count] = sTDA_fly(V[:, count:new_count])
        count = new_count

        # V_orthonormality = check_orthonormal(V[:,:count])
        # current_dic['step' + str(i)]['V_orthonormality'] = float(V_orthonormality)

    precondition_end = time.time()
    precondition_time = precondition_end - precondition_start
    if i == (max -1):
        print ('_________________ sTDA Preconditioner Failed Due to Iteration Limit _________________')
        print ('sTDA preconditioning failed after ', i, 'steps; ', round(precondition_time, 4), 'seconds')
        print ('current residual norms', Norms_of_r)
        print ('max_norm = ', max_norm)
        print ('orthonormality of V', check_orthonormal(V[:,:count]))
    else:
        print ('sTDA Preconditioning Done after ', i, 'steps; ', round(precondition_time, 4), 'seconds')

    return (full_guess*bnorm, current_dic)
###########################################################################################



#############################################
# framework of Davidson's Algorithms
###############################################################################
n = occupied*virtual

def A_diag_initial_guess (k, V):
    # m is size of subspace A matrix, also is the amount of initial guesses
    # m = min([2*k, k+8, occupied*virtual])
    m = k
    sort = hdiag.argsort()
    for j in range(m):
        V[sort[j], j] = 1.0

    return (m, V)

def sTDA_initial_guess (k, V):
    m = k
    #diagonalize sTDA_A amtrix
    V[:, :m] = Davidson0(m)

    return (m, V)
######################################################################################

#####################################################
def A_diag_preconditioner (residual, sub_eigenvalue, current_dic, current_guess=None):
    # preconditioners for each corresponding residual
    k = np.shape(residual)[1]

    t = 1e-14

    D = np.repeat(hdiag.reshape(-1,1), k, axis=1) - sub_eigenvalue
    D = np.where( abs(D) < t, np.sign(D)*t, D) # force all values not in domain (-t, t)

    new_guess = residual/D

    return new_guess, current_dic
#######################################################

################################################################################
# original simple Davidson, just to solve eigenvalues and eigenkets of sTDA_A matrix
def Davidson0 (k):

    sTDA_D_start = time.time()
    tol = 1e-4 # Convergence tolerance

    max = 30
    #################################################
    # m is size of subspace
    m = min([k+8, 2*k, n])
    V = np.zeros((n, (max+1)*k))
    W = np.zeros((n, (max+1)*k))
    # positions of hdiag with lowest values set as 1

    m, V = A_diag_initial_guess(k, V)

    W[:, :m] = sTDA_fly(V[:, :m])
    # create transformed guess vectors

    #generate initial guess and put in holders V and W
    ###########################################################################################
    for i in range(0, max):
        sub_A = np.dot(V[:,:m].T, W[:,:m])
        sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A)
        # Diagonalize the subspace Hamiltonian, and sorted.
        #sub_eigenvalue[:k] are smallest k eigenvalues
        residual = np.dot(W[:,:m], sub_eigenket[:,:k]) - np.dot(V[:,:m], sub_eigenket[:,:k] * sub_eigenvalue[:k])

        Norms_of_r = np.linalg.norm (residual, axis=0, keepdims = True)
        # largest residual norm
        max_norm = np.max(Norms_of_r)
        if max_norm < tol:
            break
        # index for unconverged residuals
        index = [i for i in range(np.shape(Norms_of_r)[1]) if Norms_of_r[0,i] > tol]
        ########################################
        # preconditioning step
        # only generate new guess from unconverged residuals
        Y = None
        new_guess, Y = A_diag_preconditioner (residual[:,index], sub_eigenvalue[:k][index], Y)
        # orthonormalize the new guesses against old guesses and put into V holder
        V, new_m = Gram_Schdmit_fill_holder (V, m, new_guess)
        W[:, m:new_m] = sTDA_fly (V[:, m:new_m])
        m = new_m
    ###########################################################################################
    full_guess = np.dot(V[:,:m], sub_eigenket[:, :k])

    sTDA_D_end = time.time()
    sTDA_D = sTDA_D_end - sTDA_D_start
    print ('sTDA A diagonalization:','threshold =', tol, '; in', i, 'steps ', round(sTDA_D, 4), 'seconds' )
    return (full_guess)
###########################################################################################


################################################################################
# Real Davidson frame, where we can choose different initial guess and preconditioner
def Davidson (k, tol, i, p, Davidson_dic):
    D_start = time.time()
    Davidson_dic['initial guess'] = i
    Davidson_dic['preconditioner'] = p
    Davidson_dic['nstate'] = k


    Davidson_dic['functional'] = mf.xc
    Davidson_dic['threshold'] = tol
    Davidson_dic['iteration'] = []
    iteration_list = Davidson_dic['iteration']

    if i == 'sTDA':
        initial_guess = sTDA_initial_guess
    elif i == 'Adiag':
        initial_guess = A_diag_initial_guess


    if p == 'sTDA':
        precondition = on_the_fly_sTDA_preconditioner
    elif p == 'Adiag':
        precondition = A_diag_preconditioner
    elif p == 'Jacobi':
        precondition = Jacobi_preconditioner
        
        
        
    print ('Initial guess:  ', i)
    print ('Preconditioner: ', p)

    print ('A matrix size = ', n,'*', n)
    max = 50
    # Maximum number of iterations



    n_initial = min([k + 8, 2 * k, n])




    #################################################
    # generate initial guess

    V = np.zeros((n, max * k + n_initial))
    W = np.zeros((n, max * k + n_initial))
    # positions of hdiag with lowest values set as 1
    # hdiag is non-interacting A matrix

    init_start = time.time()
    m, V = initial_guess(n_initial, V)
    init_end = time.time ()
    init_time = init_end - init_start

    print ('Intial guess time:', round(init_time, 4), 'seconds')
    #generate initial guess and put in holders V and W
    # m is size of subspace

    # W = Av, create transformed guess vectors
    W[:, :m] = matrix_vector(V[:, :m])

    # time cost for preconditioning
    Pcost = 0
    ###########################################################################################
    for ii in range(0, max):
        print ('Davidson', ii)

        # sub_A is subspace A matrix
        sub_A = np.dot(V[:,:m].T, W[:,:m])

        print ('subspace size: ', np.shape(sub_A)[0])

        sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A)
        # Diagonalize the subspace Hamiltonian, and sorted.
        #sub_eigenvalue[:k] are smallest k eigenvalues
        full_guess = np.dot(V[:,:m], sub_eigenket[:, :k])

        residual = np.dot(W[:,:m], sub_eigenket[:,:k]) - full_guess * sub_eigenvalue[:k]

        Norms_of_r = np.linalg.norm (residual, axis=0, keepdims = True)

        # largest residual norm
        max_norm = np.max(Norms_of_r)


        iteration_list.append({})
        current_dic = iteration_list[ii]
        current_dic['Davidosn residual norms'] = Norms_of_r[0,:].tolist()


        if max_norm < tol:
            print ('All guesses converged!')
            break

        # index for unconverged residuals
        index = [i for i in range(np.shape(Norms_of_r)[1]) if Norms_of_r[0,i] > tol]


        ########################################
        # preconditioning step
        # only generate new guess from unconverged residuals
        P_start = time.time()
        new_guess, current_dic = precondition (residual[:,index], sub_eigenvalue[:k][index], current_dic, full_guess[:,index])
        P_end = time.time()

        iteration_list[ii] = current_dic

        Pcost += P_end - P_start

        # orthonormalize the new guesses against old guesses and put into V holder
        V, new_m = Gram_Schdmit_fill_holder (V, m, new_guess)
        W[:, m:new_m] = matrix_vector (V[:, m:new_m])
        print ('preconditioned guesses:', new_m - m)
        m = new_m

    D_end = time.time()
    Dcost = D_end - D_start
    Davidson_dic['SCF time'] = kernel_t
    Davidson_dic['Initial guess time'] = init_time
    
    Davidson_dic['precondition time'] = Pcost

    Davidson_dic['Davidson time'] = Dcost
    Davidson_dic['iterations'] = ii+1
    Davidson_dic['A size'] = n
    Davidson_dic['final subspace'] = np.shape(sub_A)[0]
    Davidson_dic['excitation energy(eV)'] = (sub_eigenvalue[:k]*27.211386245988).tolist()
    ###########################################################################################
    if ii == (max -1):
        print ('============ Davidson Failed Due to Iteration Limit ==============')
        print ('Davidson failed after ', round(Dcost, 4), 'seconds')
        print ('current residual norms', Norms_of_r)
        print ('max_norm = ', max_norm)

    else:
        print ('Davidson done after ', round(Dcost, 4), 'seconds')
        print ('Total steps =', ii+1)
        print ('Final subspace shape = ', np.shape(sub_A))

    print ('Preconditioning time:', round(Pcost, 4), 'seconds')
    return (sub_eigenvalue[:k]*27.211386245988, full_guess)
################################################################################

In [72]:
combo = [['sTDA','sTDA'],['Adiag','Adiag'],['Adiag','sTDA'], ['sTDA','Adiag'], ['sTDA','Jacobi'],['Adiag','Jacobi']]
for option in [0,4,2,5]:
    i,p = combo[option]
    print ('-------------------------------------------------------------------')
    print ('|---------------   In-house Developed Davidson Starts   -----------|')

    total_start = time.time()
    Davidson_dic = {}
    Excitation_energies, kets = Davidson (5, 1e-7, i, p, Davidson_dic)
    total_end = time.time()
    total_time = total_end - total_start

    print ('In-house Davidson time:', round(total_time, 4), 'seconds')

    print ('Excited State energies (eV) =')
    print (Excitation_energies)

    print ('|---------------   In-house Developed Davidson Done   -----------|')

-------------------------------------------------------------------
|---------------   In-house Developed Davidson Starts   -----------|
Initial guess:   sTDA
Preconditioner:  sTDA
A matrix size =  960 * 960
sTDA A diagonalization: threshold = 0.0001 ; in 6 steps  0.0354 seconds
Intial guess time: 0.0365 seconds
Davidson 0
subspace size:  10
sTDA Preconditioning Done after  8 steps;  0.0184 seconds
preconditioned guesses: 5
Davidson 1
subspace size:  15
sTDA Preconditioning Done after  8 steps;  0.0169 seconds
preconditioned guesses: 5
Davidson 2
subspace size:  20
sTDA Preconditioning Done after  7 steps;  0.0183 seconds
preconditioned guesses: 5
Davidson 3
subspace size:  25
sTDA Preconditioning Done after  14 steps;  0.0297 seconds
preconditioned guesses: 5
Davidson 4
subspace size:  30
sTDA Preconditioning Done after  16 steps;  0.0362 seconds
preconditioned guesses: 5
Davidson 5
subspace size:  35
sTDA Preconditioning Done after  15 steps;  0.0231 seconds
preconditioned guesses: 5

preconditioned guesses: 5
Davidson 5
subspace size:  35
K inverse Done after  12 steps;  0.0257 seconds
K inverse Done after  10 steps;  0.021 seconds
Alpha = [2.14132464e-09 1.55484270e-08 1.03336316e-08 1.45134599e-07
 5.91721015e-08]
preconditioned guesses: 5
Davidson 6
subspace size:  40
K inverse Done after  13 steps;  0.0245 seconds
K inverse Done after  10 steps;  0.0178 seconds
Alpha = [-4.31065185e-09 -7.55556307e-10 -9.91143192e-09  1.21955015e-07]
preconditioned guesses: 4
Davidson 7
subspace size:  44
K inverse Done after  20 steps;  0.0252 seconds
K inverse Done after  19 steps;  0.0229 seconds
Alpha = [-5.13970846e-10 -2.69644336e-08]
preconditioned guesses: 2
Davidson 8
subspace size:  46
K inverse Done after  22 steps;  0.0247 seconds
K inverse Done after  21 steps;  0.0218 seconds
Alpha = [1.14162577e-08]
preconditioned guesses: 1
Davidson 9
subspace size:  47
K inverse Done after  27 steps;  0.0423 seconds
K inverse Done after  21 steps;  0.0227 seconds
Alpha = [1.255